In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce

client_df = pd.read_parquet(r'C:\Users\Eric\Documents\___Portfolio\bcg-churn\data\client_cleaned.parquet')
price_df = pd.read_parquet(r'C:\Users\Eric\Documents\___Portfolio\bcg-churn\data\price_cleaned.parquet')

In [3]:
def calculate_price_differences(price_df, price_columns):
    """
    Calculates the price differences between December and January for the given price columns.
    
    Args:
        price_df (pandas.DataFrame): DataFrame containing the price data with columns 'id', 'price_date', and the specified price columns.
        price_columns (list): List of price column names (e.g., ['price_e_1', 'price_p_1', 'price_e_2']).
    
    Returns:
        pandas.DataFrame: DataFrame with columns 'id' and the price columns suffixed with '_diff_dec_jan'.
    """
    # Group prices by companies and month
    monthly_price_by_id = price_df.groupby(['id', 'price_date'])[price_columns].mean().reset_index()

    # Get January and December prices
    jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
    dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

    # Calculate the differences
    diff = pd.merge(dec_prices.rename(columns={col: f'dec_{col}' for col in price_columns}), jan_prices.drop(columns='price_date'), on='id')
    
    # Calculate price differences for each column
    for col in price_columns:
        diff[f'{col}_diff_dec_jan'] = diff[f'dec_{col}'] - diff[col]

    # Select the id and price difference columns
    diff = diff[['id'] + [f'{col}_diff_dec_jan' for col in price_columns]]
    
    return diff

price_columns = price_df.filter(regex="price_e|price_p").columns.tolist()
price_diffs = calculate_price_differences(price_df, price_columns)

In [4]:
def aggregate_prices(price_df):
    """
    Aggregates the prices by id and calculates the mean and standard deviation.

    Args:
        price_df (pandas.DataFrame): DataFrame containing the price data with a column 'id' and various price columns.
    
    Returns:
        pandas.DataFrame: DataFrame with columns 'id', and the mean and standard deviation values per id.
    """
    agg_prices = price_df.select_dtypes(exclude='datetime')
    agg_prices = agg_prices.groupby('id').agg(['mean', 'std'])

    # Flatten the MultiIndex column names and join them with an underscore
    agg_prices.columns = ['_'.join(col).strip() for col in agg_prices.columns.values]
    agg_prices = agg_prices.reset_index()

    return agg_prices

agg_prices = aggregate_prices(price_df)


In [5]:
def count_zeros(df, price_columns):
    """
    Creates new DataFrame that represent the count of zeros in the specified columns for each id. Drops the old price columns.

    Args:
        df (pandas.DataFrame): DataFrame with price columns.
        price_columns (list): List of price column names.

    Returns:
        pandas.DataFrame: DataFrame with new columns representing counts of zeros for each id.
    """
    df_zeros = df.copy()
    df_zeros = df_zeros.select_dtypes(exclude='datetime')
    for col in price_columns:
        df_zeros[f'{col}_zero'] = (df_zeros[col] == 0).astype(int)
        
    # Sum zeros for each 'id'
    df_zeros = df_zeros.groupby('id').sum().reset_index()

    # Drop old price columns
    df_zeros = df_zeros.drop(price_columns, axis=1)

    return df_zeros


zero_df = count_zeros(price_df, price_columns)


In [6]:
def convert_to_binary(df):
    """
    Converts all columns in the DataFrame to binary variables, where 0 represents zero values and 1 represents non-zero values.
    The 'id' column is excluded from conversion.

    Args:
        df (pandas.DataFrame): DataFrame containing the columns to be converted.

    Returns:
        pandas.DataFrame: DataFrame with all columns (except 'id') converted to binary variables.
    """
    df_binary = df.copy()
    cols_to_convert = [col for col in df.columns if col != 'id']
    df_binary[cols_to_convert] = df_binary[cols_to_convert].applymap(lambda x: 0 if x == 0 else 1 if not pd.isnull(x) else np.nan)
    
    return df_binary

is_zero_df = convert_to_binary(zero_df)

In [7]:
def merge_dataframes(original_df, df_list):
    """
    Merges the provided dataframes on 'id'.
    
    Args:
        original_df (pandas.DataFrame): The original DataFrame.
        df_list (list): List of DataFrames to be merged with the original DataFrame.
        
    Returns:
        pandas.DataFrame: The merged DataFrame.
    """
    # Merge all dataframes in the list with the original dataframe
    merged_df = reduce(lambda left,right: pd.merge(left, right, on='id', how='left'), df_list, original_df)
    
    return merged_df

merged_df = merge_dataframes(client_df, [price_diffs, agg_prices, zero_df])

## Save merged dataframe

In [8]:
try:
    merged_df.to_parquet(r'C:\Users\Eric\Documents\___Portfolio\bcg-churn\data\df_merged.parquet')
    print("Files saved successfully")
except Exception as e:
    print(f"Error occurred: {e}")

Files saved successfully
